In [62]:
from __future__ import print_function
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

import matplotlib.pyplot as plt
import numpy as np

In [63]:

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..


In [64]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10, grayscale=False):
        super(LeNet, self).__init__()
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.conv1 = nn.Sequential(OrderedDict([
            ('c1',nn.Conv2d(in_channels,6,kernel_size=(5,5))),
            ('relu1',nn.ReLU()),
            ('s1',nn.MaxPool2d(kernel_size=(2,2),stride=2))
        ]))

        self.conv2 = nn.Sequential(OrderedDict([
            ('c2',nn.Conv2d(6,16,kernel_size=(5,5))),
            ('relu2',nn.ReLU()),
            ('s2',nn.MaxPool2d(kernel_size=(2,2),stride=2))
        ]))

        self.conv3 = nn.Sequential(OrderedDict([
            ('c3',nn.Conv2d(16,120,kernel_size=(5,5))),
            ('relu3',nn.ReLU()),
        ]))

        self.flatten = nn.Flatten()

        self.fc1 = nn.Sequential(OrderedDict([
            ('f5',nn.Linear(120,84)),
            ('relu5',nn.ReLU())
        ]))

        self.fc2 = nn.Sequential(OrderedDict([
            ('f6',nn.Linear(84,num_classes)),
            ('sig6',nn.LogSoftmax(dim=-1))
        ]))
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = self.flatten(x)

        x = self.fc1(x)
        out = self.fc2(x)
        return out

In [65]:
class LeNet_Dropout(nn.Module):
    def __init__(self, num_classes=10, grayscale=False):
        super(LeNet_Dropout, self).__init__()
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.conv1 = nn.Sequential(OrderedDict([
            ('c1',nn.Conv2d(in_channels,6,kernel_size=(5,5))),
            ('relu1',nn.ReLU()),
            ('s1',nn.MaxPool2d(kernel_size=(2,2),stride=2))
        ]))
        
        self.conv2 = nn.Sequential(OrderedDict([
            ('c2',nn.Conv2d(6,16,kernel_size=(5,5))),
            ('relu2',nn.ReLU()),
            ('s2',nn.MaxPool2d(kernel_size=(2,2),stride=2))
        ]))

        self.conv3 = nn.Sequential(OrderedDict([
            ('c3',nn.Conv2d(16,120,kernel_size=(5,5))),
            ('relu3',nn.ReLU()),
        ]))

        self.flatten = nn.Flatten()

        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Sequential(OrderedDict([
            ('f5',nn.Linear(120,84)),
            ('relu5',nn.ReLU())
        ]))

        self.fc2 = nn.Sequential(OrderedDict([
            ('f6',nn.Linear(84,num_classes)),
            ('sig6',nn.LogSoftmax(dim=-1))
        ]))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dropout(x) 
        x = self.fc1(x)
        out = self.fc2(x)
        return out

In [66]:
class LeNet_Normalization(nn.Module):
    def __init__(self, num_classes=10, grayscale=False):
        super(LeNet_Normalization, self).__init__()
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.conv1 = nn.Sequential(OrderedDict([
            ('c1',nn.Conv2d(in_channels,6,kernel_size=(5,5))),
            ('relu1',nn.ReLU()),
            ('BatchNorm1',nn.BatchNorm2d(6)), # Normalize After Activation
            ('s1',nn.MaxPool2d(kernel_size=(2,2),stride=2))
        ]))

        self.conv2 = nn.Sequential(OrderedDict([
            ('c2',nn.Conv2d(6,16,kernel_size=(5,5))),
            ('relu2',nn.ReLU()),
            ('s2',nn.MaxPool2d(kernel_size=(2,2),stride=2))
        ]))

        self.conv3 = nn.Sequential(OrderedDict([
            ('c3',nn.Conv2d(16,120,kernel_size=(5,5))),
            ('relu3',nn.ReLU()),
        ]))
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Sequential(OrderedDict([
            ('f5',nn.Linear(120,84)),
            ('relu5',nn.ReLU())
        ]))

        self.fc2 = nn.Sequential(OrderedDict([
            ('f6',nn.Linear(84,num_classes)),
            ('sig6',nn.LogSoftmax(dim=-1))
        ]))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        out = self.fc2(x)
        return out

In [67]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        output = model(data)

        loss = F.nll_loss(output,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [68]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [69]:
def Net():
    time0 = time.time()

    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.05
    no_cuda = False
    save_model = False

    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
        
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   

In [70]:
Net()

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.306808
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.291322
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.227820
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.141675
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.009186
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.024735
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.980713
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.970848
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.917192
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.088727
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.878537
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.838154
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.832446
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.818431
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.871033
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.821809
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.697063
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.888197
Train Epoch: 1 [23040/50000 (46%)]	Loss

In [71]:
def Net_Dropout():
    time0 = time.time()

    # Training settings
    batch_size = 128
    epochs = 10
    lr = 0.05
    no_cuda = False
    save_model = False

    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet_Dropout().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
        
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   

In [72]:
Net_Dropout()

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.302197
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.302625
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.263281
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.221774
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.110739
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.150043
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.097450
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.175186
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.991415
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.950650
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.885741
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.036981
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.820403
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.932456
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.955712
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.916000
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.892830
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.069235
Train Epoch: 1 [23040/50000 (46%)]	Loss

In [73]:
def Net_Normalization():
    time0 = time.time()

    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.05
    no_cuda = False
    save_model = False

    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet_Normalization().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
        
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))

In [74]:
Net_Normalization()

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.309083
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.285781
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.137115
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.064790
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.882430
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.969863
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.845258
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.922155
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.889429
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.880554
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.671870
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1.809917
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.539672
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1.802867
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1.888657
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.690255
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1.590110
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1.870527
Train Epoch: 1 [23040/50000 (46%)]	Loss